# Simulation

Now let's simulate and make a video. We'll use MuJoCo's main high level function `mj_step`, which steps the state $x_{t+h} = f(x_t)$.

Note that in the code block below we are *not* rendering after each call to `mj_step`. This is because the default timestep is 2ms, and we want a 60fps video, not 500fps.

This is because this model has no [degrees of freedom](https://www.google.com/url?sa=D&q=https%3A%2F%2Fen.wikipedia.org%2Fwiki%2FDegrees_of_freedom_(mechanics)) (DoFs). The things that move (and which have inertia) are called *bodies*. We add DoFs by adding *joints* to bodies, specifying how they can move with respect to their parents. Let's make a new body that contains our geoms, add a hinge joint and re-render, while visualizing the joint axis using the visualization option object `MjvOption`.

In [1]:
import mujoco
import mediapy as media
import matplotlib.pyplot as plt

import time
import itertools
import numpy as np

#model = mujoco.MjModel.from_xml_path(r"C:\Users\wenbin.li\Documents\GitHub\MuJoCo-Tutorial-fork\example_models\robotiq_2f85\scene.xml")
#model = mujoco.MjModel.from_xml_path(r"C:\Users\wenbin.li\Documents\GitHub\MuJoCo-Tutorial-fork\example_models\robotiq_2f85\2f85.xml")
model = mujoco.MjModel.from_xml_path(r"C:\Users\wenbin.li\Documents\GitHub\MuJoCo-Tutorial-fork\xml\humanoid.xml")
data = mujoco.MjData(model)
renderer = mujoco.Renderer(model)

# enable joint visualization option:
scene_option = mujoco.MjvOption()
scene_option.flags[mujoco.mjtVisFlag.mjVIS_JOINT] = True

duration = 6  # (seconds)
framerate = 60  # (Hz)

frames = []
mujoco.mj_resetData(model, data)
while data.time < duration:
  mujoco.mj_step(model, data)
  if len(frames) < data.time * framerate:
    renderer.update_scene(data, scene_option=scene_option)
    pixels = renderer.render()
    frames.append(pixels)

# Simulate and display video.
media.show_video(frames, fps=framerate)

This browser does not support the video tag.


In [2]:
model.opt.timestep

0.005

Let's change gravity and re-render:

In [3]:
print('default gravity', model.opt.gravity)
model.opt.gravity = (0, 0.5, -1)
print('changed gravity', model.opt.gravity)

frames = []
mujoco.mj_resetData(model, data)
while data.time < duration:
  mujoco.mj_step(model, data)
  if len(frames) < data.time * framerate:
    renderer.update_scene(data, scene_option=scene_option)
    pixels = renderer.render()
    frames.append(pixels)

media.show_video(frames, fps=60)

default gravity [ 0.    0.   -9.81]
changed gravity [ 0.   0.5 -1. ]


This browser does not support the video tag.


We could also have done this in XML using the top-level `<option>` element:
```xml
<mujoco>
  <option gravity="0 0 10"/>
  ...
</mujoco>
```

### Understanding Degrees of Freedom

In the real world, all rigid objects have 6 degrees-of-freedom: 3 translations and 3 rotations. Real-world joints act as constraints, removing relative degrees-of-freedom from bodies connected by joints. Some physics simulation software use this representation which is known as the "Cartesian" or "subtractive" representation, but it is inefficient. MuJoCo uses a representation known as the "Lagrangian", "generalized" or "additive" representation, whereby objects have no degrees of freedom unless explicitly added using joints.

In [8]:
print('Total number of DoFs in the model:', model.nv)
print('Generalized positions:', data.qpos)
print('Generalized velocities:', data.qvel)

Total number of DoFs in the model: 27
Generalized positions: [ 0.01175634  2.01833754  0.20035625  0.72894372 -0.67248754  0.01191608
  0.12751301 -0.06563745  0.03208637  0.00235525  0.15406243 -0.03597081
  0.11547366 -0.25504487  0.02078537 -0.00406798  0.07014585  0.07061784
 -0.07636037 -0.01625847 -0.02196082  0.16869229  0.29823624  0.56696447
 -0.39669337  0.83114749  0.10661731 -1.51989194]
Generalized velocities: [ 7.09152129e-03  3.80228721e-03 -1.10969564e-03 -6.74146292e-04
  7.21033864e-04  3.62581447e-02  3.09831080e-03  4.44874365e-03
  4.53381169e-03 -1.84840027e-04  2.60007506e-04  3.63665687e-03
  1.59299081e-02 -1.69005508e-03  7.01646241e-03  9.71400732e-03
  2.05647611e-03 -1.32391836e-02 -2.20748294e-02  9.15265011e-03
  1.68124726e-02 -1.61816982e-01  2.16635311e-01  4.46341115e-02
  1.83773114e-02 -5.31044680e-04  1.59003967e-01]


MuJoCo's use of generalized coordinates is the reason that calling a function (e.g. [`mj_forward`](https://mujoco.readthedocs.io/en/latest/APIreference.html#mj-forward)) is required before rendering or reading the global poses of objects – Cartesian positions are *derived* from the generalized positions and need to be explicitly computed.